In [5]:
import kerastuner as kt
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf
import datetime

In [6]:
# Wczytywanie danych
data = pd.read_csv('../data.csv')
numeric_columns_classification = ['Size(L)', 'OG', 'FG', 'ABV', 'IBU', 'Color', 'BoilSize', 'BoilTime', 'BoilGravity', 'Efficiency']
data[numeric_columns_classification] = data[numeric_columns_classification].apply(pd.to_numeric, errors='coerce')
data = data.dropna()
X = data[numeric_columns_classification]
y = data['Style']

# Kodowanie etykiet
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Skalowanie cech
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=(X_scaled.shape[1],)))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
tuner = kt.BayesianOptimization(build_model,
                                 objective='val_accuracy',
                                 max_trials=20, 
                                 directory='../models', 
                                 project_name='klasyfikacja_styli_piwa')

# Definiowanie callbacku EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=5, 
                               restore_best_weights=True)

log_dir = "../models/klasyfikacja_styli_piwa" #+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# Dodanie early stopping do tuner.search
tuner.search(X_scaled, y_encoded, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[early_stopping, tensorboard_callback], 
             batch_size=32)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 20 Complete [00h 00m 04s]
val_accuracy: 0.28947368264198303

Best val_accuracy So Far: 0.30921053886413574
Total elapsed time: 00h 01m 21s


In [ ]:
# Budowanie najlepszego modelu z najlepszymi hiperparametrami
best_model = build_model(best_hps)

# Wyświetlenie podsumowania najlepszego modelu
best_model.summary()

In [ ]:
# Trenowanie najlepszego modelu
history = best_model.fit(X_scaled, y_encoded, 
                         epochs=20, 
                         validation_split=0.2, 
                         callbacks=[early_stopping, tensorboard_callback],
                         batch_size=32)

final_val_accuracy = history.history['val_accuracy'][-1]
print(f"Final Validation Accuracy: {final_val_accuracy:.2f}")

In [13]:
%load_ext tensorboard
%tensorboard --logdir ../models/klasyfikacja_styli_piwa

Reusing TensorBoard on port 6008 (pid 34444), started 0:01:55 ago. (Use '!kill 34444' to kill it.)